### Post order

In [61]:
import asyncio
import xrpl
import os
from xrpl.wallet import Wallet
from xrpl.models.transactions import OfferCreate
from xrpl.transaction import submit_and_wait
from xrpl.ledger import get_latest_validated_ledger_sequence
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.account import get_next_valid_seq_number



async def place_order(seed, taker_gets, taker_pays):
    client = AsyncJsonRpcClient("https://xrplcluster.com")  # Mainnet
    client = AsyncJsonRpcClient("https://s.altnet.rippletest.net:51234")  # Testnet

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    print(wallet.classic_address)
    # Get the latest ledger index
    latest_ledger = await asyncio.to_thread(get_latest_validated_ledger_sequence, client)
    sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCreate transaction
    offer_tx = OfferCreate(
        account=wallet.classic_address,
        taker_gets=taker_gets,  # What you are selling
        taker_pays=taker_pays,  # What you want in return
        sequence=sequence,  # Ensure sequence is set
        last_ledger_sequence=latest_ledger + 5  # Valid for the next 5 ledgers
    )

    # Submit the transaction and wait for a result
    response = await asyncio.to_thread(submit_and_wait, offer_tx, client, wallet)

    # Display transaction response
    print("\nTransaction Response:")
    print(response)

    return response
     
# 🔥 Example: Selling 10 XRP for 10 USD issued by an issuer
seed = os.getenv("XRPL_ACCOUNT_SECRET")  # Replace with your real wallet seed
taker_gets = "10000000"  # Amount in drops (10 XRP = 10,000,000 drops)
taker_pays = {
    "currency": "USD",
    "issuer": "rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B",
    "value": "27.12345"  # Receiving 10 USD
}

# Run the async function
response=await place_order(seed, taker_gets, taker_pays)

rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY


C:\Users\AZUBA\anaconda3\lib\urllib\parse.py:112: RuntimeWarning: coroutine 'check_order_status' was never awaited
  return tuple(x.decode(encoding, errors) if x else '' for x in args)
C:\Users\AZUBA\anaconda3\lib\site-packages\httpcore\_backends\anyio.py:114: RuntimeWarning: coroutine 'AsyncClient.request' was never awaited
  with anyio.fail_after(timeout):



Transaction Response:
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'close_time_iso': '2025-02-12T19:00:42Z', 'ctid': 'C04915F200000001', 'hash': '7654950322BDD83EBFDF9777CB024FEFAD5B3C3C77FBD003030A0281D35FC60D', 'ledger_hash': 'C26096C019AD139E4EC3B56E1360F9C61ACC370DF913E645B44CA92FFAA8C7D8', 'ledger_index': 4789746, 'meta': {'AffectedNodes': [{'CreatedNode': {'LedgerEntryType': 'Offer', 'LedgerIndex': '38326784D9ADEB7FFB5B2C054B82102E4E1CD6EA9866575DFA47553E3266614A', 'NewFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'BookDirectory': 'DFA3B6DDAB58C7E8E5D944E736DA4B7046C30E4F460FD9DE4F09A2DD083BBA00', 'Sequence': 4732687, 'TakerGets': '10000000', 'TakerPays': {'currency': 'USD', 'issuer': 'rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B', 'value': '27.12345'}}}}, {'ModifiedNode': {'FinalFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'Balance': '99999920', 'Flags': 0, 'OwnerCount': 4, 'Sequence': 4732688}, 'LedgerEntryType': 'AccountRoot', 'LedgerIndex': '6

In [62]:
response.is_successful()

True

In [63]:
if response.is_successful():
    offer_sequence=response.result['tx_json']['Sequence']

### Check order

In [64]:
import asyncio
import os
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.wallet import Wallet
from xrpl.models.requests import AccountOffers

async def check_order_status(seed, offer_sequence):
    client = AsyncJsonRpcClient("https://xrplcluster.com")  # Mainnet
    client = AsyncJsonRpcClient("https://s.altnet.rippletest.net:51234")  # Testnet

    # Load wallet from the seed
    wallet = Wallet.from_seed(seed, algorithm="secp256k1")

    # Create the AccountOffers request
    account_offers = AccountOffers(
        account=wallet.classic_address,  # Use wallet address
        ledger_index="validated"
    )

    # Send request to XRPL
    response = await client.request(account_offers)
    result = response.result

    if "offers" in result:
        for offer in result["offers"]:
            if offer["seq"] == offer_sequence:
                print(f"🚀 Order {offer_sequence} is still open.")
                print(f"Remaining Amount: {offer['taker_gets']} | Price: {offer['taker_pays']}")
                return
        print(f"✅ Order {offer_sequence} has been **fully traded** (removed from open offers).")
    else:
        print(f"✅ Order {offer_sequence} has been **fully traded** or never existed.")

    await client.close()

# Replace with your XRP wallet secret and order sequence number
wallet_seed = os.getenv("XRPL_ACCOUNT_SECRET")  # Load from environment variable

# Run the async function
await check_order_status(wallet_seed, offer_sequence)

🚀 Order 4732687 is still open.
Remaining Amount: 10000000 | Price: {'currency': 'USD', 'issuer': 'rvYAfWj5gh67oV6fW32ZzP3Aw4Eubs59B', 'value': '27.12345'}


### Delete order

In [66]:
import asyncio
import xrpl
from xrpl.wallet import Wallet
from xrpl.models.transactions import OfferCancel
from xrpl.transaction import submit_and_wait
from xrpl.account import get_next_valid_seq_number

async def cancel_order(wallet_seed, offer_sequence):
    client = xrpl.clients.JsonRpcClient("https://xrplcluster.com")  # Mainnet
    client = xrpl.clients.JsonRpcClient("https://s.altnet.rippletest.net:51234")  # Testnet

    # Load the wallet
    wallet = Wallet.from_seed(wallet_seed, algorithm="secp256k1")  # Ensure correct key type

    # Fetch the correct sequence number
    account_sequence = await asyncio.to_thread(get_next_valid_seq_number, wallet.classic_address, client)

    # Create the OfferCancel transaction
    cancel_tx = OfferCancel(
        account=wallet.classic_address,
        offer_sequence=offer_sequence,
        sequence=account_sequence
    )

    # Submit the transaction and wait for confirmation
    response = await asyncio.to_thread(submit_and_wait, cancel_tx, client, wallet)

    # Display transaction response
    print("\nTransaction Response:")
    print(response)

# 🔥 Example: Cancel an order with sequence number 123456
wallet_seed = os.getenv("XRPL_ACCOUNT_SECRET")  # Replace with your real wallet seed


await cancel_order(wallet_seed, offer_sequence)


Transaction Response:
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'close_time_iso': '2025-02-12T19:01:51Z', 'ctid': 'C049160800000001', 'hash': '49FC5437C332F92C09E0FC745546B24B641DDD851730AB3BCA54CAC673A26A20', 'ledger_hash': 'BC682371C042E9E9A89EE9B0D7D8CBBD0C866B7B01728FB2BD38C708324442D2', 'ledger_index': 4789768, 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'Balance': '99999900', 'Flags': 0, 'OwnerCount': 3, 'Sequence': 4732690}, 'LedgerEntryType': 'AccountRoot', 'LedgerIndex': '621598FB064E9B32493510B18D16957679C5BCC6D47554A500480972D8D9F151', 'PreviousFields': {'Balance': '99999910', 'Sequence': 4732689}, 'PreviousTxnID': '037BA5A4AE17B5F1EF53394520EEF33A0644507A71BE136FE1CF2137DCE9BB65', 'PreviousTxnLgrSeq': 4789749}}], 'TransactionIndex': 0, 'TransactionResult': 'tesSUCCESS'}, 'tx_json': {'Account': 'rN1oqezvgJJpHxD4sGy5XrszBd79SugkMY', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 4789786, 'Of